# Feature Elimination

To find out which feature set is the best to use for model training, we have done the following:
- ZDEN has the highest correlation with our main targets, DTC and DTS
- here we are training models to estimate ZDEN from other features
    - we use train data for training 
    - we use test data fo testing
- we choose to always keep GR as a predictor, based on our domain knowledge
- at the end, the feature set that has lowest error on test data + ZDEN is used for our final training task on DTC and DTS
- this way, we are capturing "unknown" distruntion of the test data, which should help with model generalizability


In [ ]:
import pandas as pd
import numpy as np
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
import pickle
try:
    from src.metrics import get_metrics
except:
    from metrics import get_metrics

import itertools
import glob

cv = 10

metric = "neg_mean_squared_error" #['neg_median_absolute_error', 'neg_mean_absolute_error',
                                              #'neg_mean_squared_error', 'r2']
_null = -999
seed = 42
pre = True # for training
post = True # for evaluation
path = 'train.csv'
path_test = 'test.csv'

In [ ]:
df1 = pd.read_csv(path)
df_test = pd.read_csv(path_test)

In [ ]:
df1 = df1[list(df_test.columns)]

In [ ]:
df1.replace([str(_null), _null], np.nan, inplace=True)
df1.describe()

In [ ]:
df1.dropna(inplace=True)
df1.describe()

In [ ]:
df_test.replace([str(_null), _null], np.nan, inplace=True)
df_test.describe()

In [ ]:
df_test.dropna(inplace=True)
df_test.describe()

In [ ]:
drop_col = ['DTC', 'DTS']

In [ ]:
targets = ['ZDEN']
must_keep = ['GR']
features = [x for x in df1.columns if (x not in targets) and (x not in drop_col)]
target = targets[0]
print(features, target)

In [ ]:
train_dta = df1[features+[target]]
test_dta = df_test[features+[target]]

In [ ]:
fea_combination = []
features_iter = list(set(features) - set(must_keep))
for L in range(0, len(features_iter)+1):
    for subset in itertools.combinations(features_iter, L):
        subset = list(subset)
        subset.append(must_keep[0])
        fea_combination.append(subset)

fea_combination.remove(must_keep)

In [ ]:
target = 'ZDEN'

for count, features in enumerate(fea_combination):

    print(features)
    big_dict = {}

    X_train, y_train = train_dta[features], train_dta[target]
    X_test, y_test = test_dta[features], test_dta[target]
    tpot = TPOTRegressor(generations=3, population_size=25, verbosity=2, random_state=seed, cv=cv, periodic_checkpoint_folder='./intermediate_results_'+str(count)+'/')

    tpot.fit(X_train, y_train)
    tpot_best_model = tpot.fitted_pipeline_

    y_test_pred = tpot_best_model.predict(X_test)
    y_train_pred = tpot_best_model.predict(X_train)

    train_metric = get_metrics(y_train.values, y_train_pred)
    test_metric = get_metrics(y_test.values, y_test_pred)
    metrics = pd.DataFrame({"Metric": train_metric.index, "Train": train_metric.values, "Test": test_metric.values})
    big_dict[count] = (features, metrics)

    with open(str(count)+'_result.pickle',"wb") as fp:
        pickle.dump(big_dict, fp)

# Post-process
Here we combine all the individual metrics, and sort them based on the score on test data, the top row indicates feature set that best generalizes to the test data and we choose this set + ZDEN for our main training task on DTC and DTS.

In [ ]:
l = glob.glob('*.pickle')
big_dictionary = {}
for fname in l:
    with open(fname,"rb") as fp:
        d = pickle.load(fp)

    big_dictionary[list(d.keys())[0]] = list(d.values())[0]

In [ ]:
big_dictionary

In [ ]:
metricid = 1 #root_mean_squared_error
X = list(big_dictionary.keys())
y_train = []
y_test = []
FEA = []
for x in big_dictionary.keys():
    y_train.append(big_dictionary[x][1].iloc[metricid][1])
    y_test.append(big_dictionary[x][1].iloc[metricid][2])
    FEA.append(big_dictionary[x][0])

In [ ]:
df_comp = pd.DataFrame({'feature':FEA, 'train':y_train, 'test':y_test, 'orignal index':X})
df_comp.sort_values(by=['test'])